In [2]:
# TODO: a room with walls
# a camera that can fly
import filepath
# from panda3d.core import PointLight, DirectionalLight
from direct.task import Task
import numpy as np
import torch
from torch import Tensor
from geom.basic import create_cube_node, create_sphere_node,uv_curve_surface, create_colored_cube_node
from panda3d.core import (
    Geom,
    GeomNode,
    GeomTriangles,
    GeomVertexData,
    GeomVertexFormat,
    GeomVertexWriter,
    GeomEnums,
    NodePath,
    PointLight,
    DirectionalLight,
    CardMaker,
    WindowProperties
)
from direct.showbase import DirectObject
from panda3d_game.game_object import GameObject
from panda3d_game.controller import PlayerController

# controller
# player controller
# agent controller
from typing import Set, List, Dict,Callable
from datetime import datetime
from abc import ABC
from panda3d.core import PNMImage, Texture
from panda3d.core import PTAUchar
from panda3d.core import CardMaker
from panda3d.core import Point2
from panda3d.core import NodePath, Camera, PerspectiveLens
# import gizeh as gz
import random

In [3]:
import sympy
import torch

In [4]:
from py_src.demos.physics_room import PhyscRoom

In [5]:
from panda3d.core import Point3, LVector3f
from panda3d.bullet import (
    BulletWorld, 
    BulletBoxShape, 
    BulletPlaneShape,
    BulletRigidBodyNode, BulletSphereShape
)
import sympy as sp
from sympy.physics.units import (
    kilometer, meter,centimeter,
    gram, kilogram, tonne, 
    newton, second
)
from typing import List
from torch.nn import functional as F

In [6]:
# GameObject
#   .totol_mass

tmoon = Texture()
tmoon.read("res/moon.jpeg")
tmoon.setWrapU(Texture.WM_repeat)
tmoon.setWrapV(Texture.WM_repeat)

G_val = 6.67430*1e-11 * newton * meter**2 / (kilogram**2)
def getG(unit,G_val=G_val):
    G = sp.symbols('G')
    m1, m2 = sp.symbols('m1 m2')
    r = sp.symbols('r')
    M1 = m1 * unit["mass"]
    M2 = m2 * unit["mass"]
    R = r * unit["length"]
    F = G*(M1*M2)/R**2
    f = F/unit["force"]
    G_game = f.simplify().subs({
        m1:1,
        m2:1,
        r:1,
        G:G_val
    })
    return G_game
def safe_reciprocal(x):
    return torch.where(x==0, 0*x, torch.reciprocal(x))

one = sp.Number(1)

class MassedBall(): # not yet inherent GameObject
    def __init__(
        self,
        name,
        radius,
        mass,
        units:dict
        # lat_res,
        # lon_res,
        # geom_type
        # TODO: texture
    ):
        # super().__init__(name)
        self.worlds = []
        self.radius = radius
        self.mass = mass
        self.units = units
        # print(radius / units["length"])
        self.game_radius:float = (radius / units["length"]).simplify().evalf()
        self.game_mass: float = (mass / units["mass"]).simplify().evalf()
        # create geom
        self.geom_node = create_sphere_node(name, lat_res=12,lon_res=12)
        self.geom_np = NodePath(self.geom_node)
        # print(self.game_radius)
        self.geom_np.setScale(self.game_radius)
        # create rigid body
        self.rigid_body_node = BulletRigidBodyNode("SphrRg."+str(name))
        sphere_shape = BulletSphereShape(self.game_radius)
        self.rigid_body_node.add_shape(sphere_shape)
        self.rigid_body_node.set_mass(self.game_mass)
        self.rigid_body_np = NodePath(self.rigid_body_node)
        # bind geom to rigid body
        # self.rigid_body_np.attachNewNode(self.geom_node)
        self.geom_np.reparent_to(self.rigid_body_np)
        # bind self to rigid body
        # self.reparent_to(self.rigid)
        # self.geom_np.reparentTo(self)
        # todo: add collison
        
    
    def toBulletWorld(self, world:BulletWorld):
        self.worlds.append(world)
        world.attach_rigid_body(self.rigid_body_node)
        # if there is contraint, attach constraint
        
    def set_texture(self, t):
        self.geom_np.set_texture(t)
        
    def set_tex_scale(self, t):
        self.geom_np.set_tex_scale(t)
        
    def setPos(self, *args):
        # super().setPos(self,*args)
        self.rigid_body_np.setPos(*args)
    
    @property
    def mainPath(self):
        return self.rigid_body_np
        
    def getPos(self, other=None):
        if other is None:
            return self.rigid_body_np.getPos()
        elif isinstance(other, NodePath): 
            return self.rigid_body_np.getPos(other)
        else:
            return self.rigid_body_np.getPos(other.mainPath)
        
    def reparentTo(self, *args):
        self.rigid_body_np.reparentTo(*args)
    def reparent_to(self, *args):
        self.rigid_body_np.reparent_to(*args)
    
    def setZ(self,*args):
        self.rigid_body_np.setZ(*args)
        
    def apply_force(self, force, pos):
        # print(force, pos)
        self.rigid_body_node.apply_force(LVector3f(*force), LVector3f(*pos))

def autocomplete_units(unit:dict):
    force_unit = (
        unit["mass"] * unit["length"] / unit["time"] ** 2
    )/(kilogram * meter / second**2).simplify()
    unit["force"] = force_unit * newton

class PhysicsRoomBalls(PhyscRoom):
    @property
    def masses(self):
        if hasattr(self, "_masses"):
            return self._masses
        else:
            self._masses =  self.get_mass(self.gravitational_bodies)
            return self._masses
    def __init__(self, xb:int, yb:int , zb:int):
        super().__init__(xb,yb,zb)
        # objects
        
        self.bullet_world.setGravity((0,0,0)) # no global gravity
        self.unit = {
            "mass" : tonne,
            "length" : 100*meter,
            "time": 1 * second, 
            # "force" : sp.Number(1e3) * newton
        }
        autocomplete_units(self.unit)
        # self.G_game = getG(self.unit, G_val=G_val)
        self.G_game = 0.001
        self.G_game_tensor = torch.Tensor([self.G_game])
        
        self.planet1 = MassedBall(
            name="planet1",
            radius=100*meter,
            mass=1e6*tonne,
            units=self.unit
        )
        self.planet1.reparentTo(self.render)
        self.planet2 = MassedBall(
            name="planet2",
            radius=100*meter,
            mass=2e6*tonne,
            units=self.unit
        )
        self.planet2.reparentTo(self.render)
        self.planet2.toBulletWorld(self.bullet_world)
        
        
        self.planet1.set_texture(tmoon)
        self.planet2.set_texture(tmoon)
        self.planet2.setZ(4)
        self.planet1.setZ(-4)
        self.planet1.toBulletWorld(self.bullet_world)
        self.gravitational_bodies = [self.planet1, self.planet2]
        
        print(self.masses)
        dist = self.get_node_dist(self.gravitational_bodies)
        gravity = self.cal_gravity(self.masses, dist)
        print(gravity)
        
    
    def apply_gravitational_force(self, task):
        # N by N matrix, upper triaglar
        gravity = self.cal_gravity(
            self.masses,
            self.get_node_dist(self.gravitational_bodies),
            sumup=True
        )
        self.batch_apply_force(
            self.gravitational_bodies,
            gravity,
            self.get_node_poses(self.gravitational_bodies)
        )
        
    @staticmethod
    def get_node_poses(node_paths:List[NodePath]) -> Tensor:
        pos = torch.Tensor([
            n.getPos()
            for n in node_paths
        ])
        return pos
    
    @staticmethod
    def get_node_dist(node_paths:List[NodePath]) -> Tensor:
        nr_nodes = len(node_paths)
        # FIXME: use vector distance
        # dist ij
        return torch.Tensor([
            [
                node_paths[j].getPos(node_paths[i])
                # if i<j else (0,0,0)
                for j in range(nr_nodes)
            ]
            for i in range(nr_nodes)
        ])
    
    @staticmethod
    def get_mass(objs:List[MassedBall]) -> torch.Tensor:
        return torch.Tensor([
            o.game_mass for o in objs
        ])
    
    def cal_gravity(self, mass:torch.Tensor, dist:torch.Tensor, sumup=False) -> torch.Tensor:
        dist_sq = torch.einsum(
            "mnd -> mn",
            dist ** 2
        )
        dist_sq_rcpr = safe_reciprocal(dist_sq)
        gravity_scalar = torch.einsum(
            "o, n, mn, m -> mn",
            self.G_game_tensor, 
            mass, dist_sq_rcpr, mass
        )
        gravity_vector = torch.einsum(
            "mn, mnd -> mnd",
            gravity_scalar, 
            F.normalize(dist, dim=2, p=2)
        )
        if sumup:
            return torch.einsum(
                "mnd -> md",
                gravity_vector
            )
        return gravity_vector
        
    @staticmethod
    def batch_apply_force(bodies, force, pos):
        for i in range(len(bodies)):
            bodies[i].apply_force(
                force[i],
                pos[i]
            )
            
    def update(self, task):
        if not self.paused:
            self.apply_gravitational_force(task)
        return super().update(task)
    
    
        
import builtins
import traceback
try:
    with PhysicsRoomBalls(25,25,25) as app:
        app.run()
except Exception as e:
    print(e)
    print(traceback.format_exc())
finally:
    if hasattr(builtins, 'base'):
        builtins.base.destroy()             

init context showbase


Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager


---set ref---:render,<class 'panda3d.core.NodePath'>
tensor([1000000., 2000000.])
tensor([[[        0.,         0.,         0.],
         [        0.,         0.,  31250002.]],

        [[        0.,         0., -31250002.],
         [        0.,         0.,         0.]]])
---Physics Room run(), at 2024-09-06 23:42:40.863694---
---Physics Room destroy at 2024-09-06 23:42:44.796890---
---Physics Room destroyed at 2024-09-06 23:42:44.797355, exit---


SystemExit: 

/media/ywatcher/ExtDisk1/LDisk/Packages/game_env/env/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [9]:
class PhysicsRoomPlane(PhyscRoom):
    def __init__(self, xb:int, yb:int , zb:int):
        super().__init__(xb,yb,zb)
        # objects
        
        self.bullet_world = BulletWorld()
        self.bullet_world.setGravity((0,0,-9.81))
        self.groundNode = BulletRigidBodyNode('ground')
        ground = BulletPlaneShape((0,0,1),1)
        self.groundNode.addShape(ground)
        self.groundNp = self.render.attachNewNode(self.groundNode)
        self.bullet_world.attach_rigid_body(self.groundNode)
        self.groundNp.setPos(0,0,-zb)
        
        self.sphereNode = BulletRigidBodyNode('sphere')
        sphere_rigid = BulletSphereShape(1)
        self.sphereNode.setMass(1)
        self.sphereNode.addShape(sphere_rigid)
        # self.sphereNp = self.render.attachNewNode(self.sphereNode)
        self.sphereNp = NodePath(self.sphereNode)
        self.sphereNp.reparentTo(self.render)
        sphere_geom = create_sphere_node('sphere01',lat_res=6,lon_res=6)
        sphere_geom_np = NodePath(sphere_geom)
        sphere_geom_np.reparentTo(self.sphereNp)
        sphere_geom_np.setPos(0,0,2)
        self.bullet_world.attach_rigid_body(self.sphereNode)
        sphere_geom_np.setTexture(tmoon)
        # you cannot use non physical parent nodepath to
        # set rigid pos
        # self.new_np = NodePath("hey")
        # self.sphereNp.reparent_to(self.new_np)
        # self.new_np.reparent_to(self.render)
        # self.sphereNp.setPos(0,0,2)
        # self.new_np.setPos(0,0,2)
        
    
        
import builtins
import traceback
try:
    with PhysicsRoomPlane(25,25,25) as app:
        app.run()
except Exception as e:
    print(e)
    print(traceback.format_exc())
finally:
    if hasattr(builtins, 'base'):
        builtins.base.destroy()             

Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager


---set ref---:render,<class 'panda3d.core.NodePath'>
---Physics Room run(), at 2024-08-27 13:00:03.180302---
paused:False
---Physics Room destroy at 2024-08-27 13:00:15.826385---
---Physics Room destroyed at 2024-08-27 13:00:15.826845, exit---


SystemExit: 

In [105]:
autocomplete_units(unit)
# unit

In [ ]:
# model
# either reparent to some rigid body
# or a node, the position of node updates according to a function

# geom nodes categorized into groups
# each group renders one geom in certain condition
# rendership do not change each frame
# change it when certain criteria met

In [31]:
# calculate gravity

In [33]:

F = G*(m1*m2)/r**2
F

G*m1*m2/r**2

In [34]:
unit = {
            "mass" : tonne,
            "length" : 100*meter,
            "time": 1 * second, 
            # "force" : sp.Number(1e3) * newton
        }
autocomplete_units(unit)

In [35]:
unit

{'mass': tonne,
 'length': 100*meter,
 'time': second,
 'force': 100*newton*tonne/kilogram}

In [42]:
G_val = 6.67430*1e-11 * newton * meter**2 / (kilogram**2)
M1 = m1 * unit["mass"]
M2 = m2 * unit["mass"]
R = r * unit["length"]
F = G*(M1*M2)/R**2
f = F/unit["force"]
G_game = f.simplify().subs({
    m1:1,
    m2:1,
    r:1,
    G:G_val
})

In [43]:
G_game

6.67430000000000e-14

In [78]:
x = torch.Tensor([[1,0],[0,0]])*torch.Tensor([one])


In [72]:
type(x.to(dtype=float)[0].item())

float

In [73]:
type(torch.reciprocal(x)[0].item())

float

In [76]:
torch.Tensor([one])

tensor([1.])

In [70]:
one = sp.Number(1)

In [80]:
F.normalize(x, dim=-1,p=2)

tensor([[1., 0.],
        [0., 0.]])

In [88]:
x.numpy()

array([[1., 0.],
       [0., 0.]], dtype=float32)

In [90]:
y = torch.Tensor([1,2,3])

In [93]:
LVector3f(*y)

LVector3f(1, 2, 3)